In [ ]:
#this notebook will used for in purpose of feature embeding into LSTM and GRU layers

In [ ]:
from google.colab import drive 
drive.mount('/content/drive')

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 

import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout, LSTM, GRU, Embedding, Flatten, Lambda, Multiply, Concatenate
from tensorflow.keras.preprocessing. text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from platform import python_version
import os
import sys
from sklearn.model_selection import train_test_split

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
a_col = ['anime_id', 'title', 'title_english', 'title_japanese',
       'title_synonyms', 'image_url', 'type', 'source', 'episodes', 'status',
       'airing', 'aired_string', 'aired', 'duration', 'rating', 'score',
       'scored_by', 'rank', 'popularity', 'members', 'favorites', 'background',
       'premiered', 'broadcast', 'related', 'producer', 'licensor', 'studio',
       'genre', 'opening_theme', 'ending_theme', 'duration_min',
       'aired_from_year']
b_col = ['username', 'anime_id', 'my_watched_episodes', 'my_start_date',
       'my_finish_date', 'my_score', 'my_status', 'my_rewatching',
       'my_rewatching_ep', 'my_last_updated', 'my_tags']
c_col = ['username', 'user_id', 'user_watching', 'user_completed', 'user_onhold',
       'user_dropped', 'user_plantowatch', 'user_days_spent_watching',
       'gender', 'location', 'birth_date', 'access_rank', 'join_date',
       'last_online', 'stats_mean_score', 'stats_rewatched', 'stats_episodes']
NROWS = 1000

In [ ]:
b = pd.read_csv('drive/MyDrive/Colab Notebooks/datasets/anime/animelists_cleaned.csv',
                # nrows = NROWS,
                usecols = ['username', 'anime_id','my_score'])

c = pd.read_csv('drive/MyDrive/Colab Notebooks/datasets/anime/users_cleaned.csv',
                # nrows = NROWS,
                usecols = ['username', 'user_id'])

df_long = c.merge(b, left_on = 'username', right_on = 'username')
df_long = df_long.drop(columns = ['username'])
df, df_test = train_test_split(df_long, train_size = 0.025, test_size = 0.975, random_state  =42)
df = df.reset_index(drop = True)
df = df.rename(columns={'user_id':'user', 'anime_id':'item', 'my_score':'plays'})

In [ ]:
def get_negatives(uids, iids, items, df_test):
    """Returns a pandas dataframe of 100 negative interactions
    based for each user in df_test.
    Args:
        uids (np.array): Numpy array of all user ids.
        iids (np.array): Numpy array of all item ids.
        items (list): List of all unique items.
        df_test (dataframe): Our test set.
    Returns:
        df_neg (dataframe): dataframe with 100 negative items 
            for each (u, i) pair in df_test.
    """

    negativeList = []
    test_u = df_test['user_id'].values.tolist()
    test_i = df_test['item_id'].values.tolist()

    test_ratings = list(zip(test_u, test_i))
    zipped = set(zip(uids, iids))

    for (u, i) in test_ratings:
        negatives = []
        negatives.append((u, i))
        for t in range(100):
            j = np.random.randint(len(items)) # Get random item id.
            while (u, j) in zipped: # Check if there is an interaction
                j = np.random.randint(len(items)) # If yes, generate a new item id
            negatives.append(j) # Once a negative interaction is found we add it.
        negativeList.append(negatives)

    df_neg = pd.DataFrame(negativeList)

    return df_neg

In [ ]:
def load_dataset(df):

    # Drop any rows with empty cells or rows
    # with a play count of zero.
    df = df.dropna()
    df = df.loc[df.plays != 0]

    # Remove any users with fewer than 1 interaction. 
    df_count = df.groupby(['user']).count()
    df['count'] = df.groupby('user')['user'].transform('count')
    df = df[df['count'] > 1]

    # Convert artists names into numerical IDs
    df['user_id'] = df['user'].astype("category").cat.codes
    df['item_id'] = df['item'].astype("category").cat.codes

    # Create a lookup frame so we can get the artist
    # names back in readable form later.
    item_lookup = df[['item_id', 'item']].drop_duplicates()
    item_lookup['item_id'] = item_lookup.item_id.astype(str)

    # Grab the columns we need in the order we need them.
    df = df[['user_id', 'item_id', 'plays']]

    # Create training and test sets.
    df_train, df_test = train_test_split(df,train_size = 0.8, test_size = 0.2, random_state = 42)

    # Create lists of all unique users and artists
    users = list(np.sort(df.user_id.unique()))
    items = list(np.sort(df.item_id.unique()))

    # Get the rows, columns and values for our matrix.
    rows = df_train.user_id.astype(int)
    cols = df_train.item_id.astype(int)

    values = list(df_train.plays)

    # Get all user ids and item ids.
    uids = np.array(rows.tolist())
    iids = np.array(cols.tolist())

    # Sample 100 negative interactions for each user in our test data
    df_neg = get_negatives(uids, iids, items, df_test)

    return uids, iids, df_train, df_test, df_neg, users, items, item_lookup

In [ ]:
uids, iids, df_train, df_test, df_neg, users, items, item_lookup = load_dataset(df) 

In [ ]:
num_neg = 3
def get_train_instances():
     """Samples a number of negative user-item interactions for each
     user-item pair in our testing data.
     Returns:
         user_input (list): A list of all users for each item
         item_input (list): A list of all items for every user,
             both positive and negative interactions.
         labels (list): A list of all labels. 0 or 1.
     """

     user_input, item_input, labels = [],[],[]
     zipped = set(zip(uids, iids))

     for (u, i) in zip(uids,iids):
         # Add our positive interaction
         user_input.append(u)
         item_input.append(i)
         labels.append(1)

         # Sample a number of random negative interactions
         for t in range(num_neg):
             j = np.random.randint(len(items))
             while (u, j) in zipped:
                 j = np.random.randint(len(items))
             user_input.append(u)
             item_input.append(j)
             labels.append(0)

     return user_input, item_input, labels

In [ ]:
user_input, item_input, labels =  get_train_instances()

In [ ]:
user_num = np.max(user_input)+2
item_num = np.max(item_input)+2
anime_num = item_num
embed_dim = 16

In [ ]:
def total_model():
  user_input = tf.keras.Input(shape = (1), name = 'user_input')
  anime_input = tf.keras.Input(shape  = (1), name  ='anime_input')

  mlp_user_emb = Embedding(user_num+2,embed_dim,name='mlp_user_embeddings')(user_input)
  mlp_item_emb = Embedding(anime_num+2,embed_dim,name='mlp_anime_embeddings')(anime_input)

  gmf_user_emb = Embedding(user_num+2,5,name='gmf_user_embeddings')(user_input)
  gmf_item_emb = Embedding(anime_num+2,5,name='gmf_anime_embeddings')(anime_input)

  # user_emb_layer = Embedding(user_num+2,embed_dim,name='user_embeddings')(user_input)
  # anime_emb_layer = Embedding(anime_num+2,embed_dim,name='anime_embeddings')(anime_input)

  mlp_flat_user = Flatten(name = 'mlp_user_flatten')(mlp_user_emb)
  mlp_flat_anime = Flatten(name = 'mlp_item_flatten')(mlp_item_emb)

  gmf_flat_user = Flatten(name = 'gmf_user_flatten')(gmf_user_emb)
  gmf_flat_anime = Flatten(name = 'gmf_anime_flatten')(gmf_item_emb)


  # print(flat_user.shape)
  # print(flat_anime.shape)
### MLP SIDE
# concat_layer = Concatenate(axis=2, name = 'concat_layer'
#                 )([user_emb_layer,anime_emb_layer])
  concat_layer = Concatenate(axis=1, name = 'concat_layer'
                )([mlp_flat_user,mlp_flat_anime])
  print(concat_layer.shape)
  mlp_layer_1 = Dense(128, activation='relu',name ='dense_1')(concat_layer)
  mlp_layer_1 = Dropout(0.5)(mlp_layer_1)
  mlp_layer_2 = Dense(64, activation='relu',name ='dense_2')(mlp_layer_1)
  mlp_layer_2 = Dropout(0.5)(mlp_layer_2)
  mlp_layer_3 = Dense(32, activation='relu',name ='dense_3')(mlp_layer_2)
  mlp_layer_3 = Dropout(0.5)(mlp_layer_3)
  mlp_layer_4 = Dense(16, activation='relu',name ='dense_4')(mlp_layer_3)
  mlp_layer_5 = Dense(8, activation='relu',name ='dense_5')(mlp_layer_4)

  print(mlp_layer_5.shape)
### MLP SIDE
### MLP SIDE
  gmf_layer = Multiply( name = 'element_multip_layer'
                )([gmf_flat_user,gmf_flat_anime])
  print('gmf_layer_shape',gmf_layer.shape)
### MLP SIDE
  neuMF_layer =Concatenate(axis=1, name = 'neuMF_layer'
                )([gmf_layer,mlp_layer_5])

  neuMF_layer_2 = Dense(8, activation='relu',name ='dense_neuMF_1')(neuMF_layer)
  neuMF_layer_3 = Dense(1, activation='sigmoid',name ='dense_neuMF_2')(neuMF_layer_2)
  
  return tf.keras.Model(inputs = [user_input, anime_input], outputs=neuMF_layer_3  )

mlp_model = total_model()
mlp_model.compile(
    loss = 'binary_crossentropy',
    # optimizer = tf.keras.optimizers.RMSprop(learning_rate = 0.001),
    optimizer = 'Adam',
    # metrics = 'accuracy',
    metrics = tf.keras.metrics.TopKCategoricalAccuracy(
    k=5, name='top_k_categorical_accuracy', dtype=None
)
)

# cat_score = to_categorical(train_df['score'])
# cat_score_test = to_categorical(test_df['score'])

mlp_hist = mlp_model.fit([
                          tf.constant(user_input), 
                          tf.constant(item_input)],
                          tf.constant(labels),
                          # train_df['score'],
                          # cat_score,
                          epochs = 1,
                          validation_split = 0.2,
                         batch_size =30)

(None, 32)
(None, 8)
gmf_layer_shape (None, 5)
39494/39494 [==============================] - 736s 19ms/step - loss: 0.4314 - top_k_categorical_accuracy: 1.0000 - val_loss: 0.4156 - val_top_k_categorical_accuracy: 1.0000


In [ ]:
df_neg

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,"(65875, 4903)",971,5440,4428,5138,5184,5015,5681,387,5127,1009,3915,2517,2487,1393,4729,2023,2654,5092,798,4685,112,1632,146,5162,540,3974,3311,2403,4968,964,1783,4142,757,1783,1378,585,1043,5697,4162,...,3727,3105,30,969,4339,2434,3585,4135,2188,5416,5792,2234,1017,3612,782,4391,3345,4807,4726,1067,5073,1274,5580,2752,4454,5706,4495,4473,3826,2627,3400,2598,2539,2867,4853,2470,978,1998,174,181
1,"(18755, 4034)",4008,4951,3388,4618,4711,2084,2388,1890,5186,5065,1275,3893,2493,1268,2531,1681,3194,5356,5744,5421,2627,1723,509,4326,3285,3555,85,1856,4546,3028,97,3699,4682,3688,495,3157,4401,501,1186,...,5019,4679,1242,2620,2662,4411,3889,2380,1898,956,49,737,3134,1839,667,2507,1166,4709,2616,5225,583,3266,3704,5306,5247,548,1356,3565,1558,1477,2855,1824,382,2505,4426,5013,5537,3129,4885,5611
2,"(9553, 79)",3813,2060,1047,5042,4243,3216,822,2049,3735,3863,3606,920,5518,4909,1190,1550,2600,2515,927,1009,586,2343,2507,5539,4909,131,3281,3793,1977,3346,4537,1127,2064,2780,2967,4743,738,2894,5714,...,43,2878,2360,3508,1326,2832,4320,2684,2076,5423,796,5658,4995,2830,3106,5692,2397,3497,5365,3439,1119,2619,5579,2952,2186,4132,5066,5623,3712,729,1672,5513,4283,1424,5619,3426,5413,1408,3679,1545
3,"(35600, 3061)",1531,556,3173,121,5523,33,4197,1818,5091,5613,4736,113,2095,5616,1389,3935,5516,3510,3912,4948,4359,4495,2803,2364,4694,317,3033,3088,2450,1883,1987,3587,5589,215,614,4881,2324,3251,3308,...,3019,1756,3141,5330,1763,4163,1778,1291,2599,604,1485,794,5315,5115,4918,5761,44,4418,4195,11,286,3528,1900,3619,5378,4340,3014,5354,286,190,3727,3256,5373,2331,359,4387,4238,1266,3889,917
4,"(3512, 1131)",4920,3287,5591,4990,1401,2675,1859,2293,3044,3361,5079,1502,3975,3001,1973,4331,4557,711,5355,4309,1447,212,2901,2530,21,3310,5606,3317,3716,4946,3053,4966,4899,244,4646,5093,4340,5502,2771,...,1164,5575,3221,1961,471,495,2427,2206,3924,2593,2933,3964,3375,2478,3127,4339,588,4422,2320,4011,217,813,5274,5701,1301,3126,3974,3454,1609,1648,4219,4954,5366,2069,3779,3270,4150,1784,758,3369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92559,"(1579, 1759)",969,4686,3477,1176,2980,3671,5164,4037,4944,2345,3596,893,4790,5372,5076,1338,1904,3213,233,2379,4468,3890,1567,2059,3900,4116,389,4155,621,5049,2186,1072,342,2543,105,250,5488,773,222,...,505,4746,4845,786,3967,343,406,3243,627,1207,1474,2041,6,3225,1806,4105,3501,3824,2428,5507,5466,5552,5005,2170,4680,1078,3365,2431,3603,4685,3605,806,4272,932,2705,3074,1778,1797,3962,4241
92560,"(19612, 4188)",604,4705,951,1200,98,1635,5621,3101,2155,4787,13,3537,1361,2810,2789,3903,807,1254,4650,3569,5616,2766,3688,2925,2630,1388,1287,1650,5432,723,2286,443,3181,3379,1172,2118,3539,1370,2173,...,693,1227,4322,5571,4129,2374,3752,4331,799,992,5545,1152,3470,1326,2155,738,4213,5782,4022,5174,5509,859,2059,418,4186,1246,4938,1709,1078,5624,1225,4547,4092,1492,5448,1899,311,3470,3043,4160
92561,"(62533, 4145)",1454,5261,410,3582,5639,3841,2113,4975,1107,3047,2098,3576,976,5247,3363,2160,2481,5687,1757,1793,4166,760,2554,3542,5703,1853,1220,4625,1893,1724,4231,2330,4224,5274,5629,3902,4852,2621,3365,...,5118,3728,4868,3457,4057,1462,3964,76,1804,5419,2562,2301,1970,74,2664,2923,1073,3190,2389,305,4493,5701,4157,1443,3805,165,5053,276,5370,1557,2431,469,3981,1378,80,4849,3725,1211,934,3541
92562,"(44766, 4308)",4121,929,1205,2557,5384,2021,553,1172,2700,5492,339,1755,4157,5280,574,52,3972,1684,4519,1839,5580,320,4462,1336,3022,3390,3452,2557,1275,258,3508,4809,4983,2966,5332,4312,1945,4638,5397,...,4

In [1]:
mlp_model.predict([3,4])

NameError: ignored